In [2]:
import pandas as pd
import os
from multiprocessing import Process
import copy
from datetime import datetime, timedelta

# Node의 부모 Class
class Node:
    def __init__(self, node_name):
        self.node_name = node_name

# Edge의 부모 Class
class Edge:
    def __init__(self, origin, dest):
        self.origin = origin
        self.dest = dest

# Airport Class
class Airport(Node):
    def __init__(self, airport_name):
        super().__init__(airport_name)

# Flight Class
class Flight(Edge):
    def __init__(self, index, origin, dest, origin_date, origin_time, dest_date, dest_time, flight_duration, flight_type):
        super().__init__(origin, dest)
        self.index = index
        self.origin_date = origin_date
        self.origin_time = origin_time
        self.dest_date = dest_date
        self.dest_time = dest_time
        self.flight_duration = flight_duration
        self.flight_type = flight_type

    def print(self):
        print(self.index, self.origin, self.origin_date, self.origin_time, self.dest,
              self.dest_date, self.dest_time, self.flight_duration, self.flight_type)

# Stack Node의 구조
class Stackitem:
    def __init__(self, flight: Flight, flights_sequence: list = None, landing_number = None):
        if flights_sequence:
            self.flight = flight
            flights_sequence.append(flight.index)
            self.flights_sequence = flights_sequence
            self.landing_number = landing_number + 1
            # self.time = time + dt.timedelta(flight.flight_duration)
        else:
            self.flight = flight
            self.flights_sequence = [flight.index]
            self.landing_number = 1
            # self.time = flight.flight_duration;
            
class Flight_subset(Flight) : 
    def __init__(self, index:str, flight : Flight) :
        super().__init__(index, flight.origin, flight.dest, flight.origin_date, flight.origin_time, flight.dest_date, flight.dest_time, flight.flight_duration, flight.flight_type)
        self.subset = [flight]
    
    def append(self, flight : Flight) :
        self.subset.append(flight) 
        

def getMinute(year, month, day, hour, minute) :
    return year * 525600 + month * 43200 + day * 1440 + hour * 60 + minute

In [15]:
# Initial Setting - Test
flight_graph = dict()
flight_excel = pd.read_excel("Real_Input_data.xlsx")
flight_excel.columns = ['INDEX', 'ORIGIN', 'ORIGIN_DATE','ORIGIN_TIME', 'DEST','DEST_DATE', 'DEST_TIME', 'FLIGHT_DURATION','AIRCRAFT_TYPE' ]
airports_set = set()

c:\Users\HyungJoon\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [19]:
# Make Flight Subset
max_row_number = 1000

visited = list()
for i in range(max_row_number) :
    visited.append(False)

flight_subset_list = list()
for i in range(max_row_number) :
    # 방문 했다면 넘어감 
    if (visited[i]) :
        continue
    
    visited[i] = True
    # 방문하지 않았다면 subset 만들기
    flight_temp_subset = Flight_subset("FS" + str(i + 1), Flight(flight_excel.loc[i]['INDEX'], flight_excel.loc[i]['ORIGIN'], flight_excel.loc[i]['DEST'],
                                                              flight_excel.loc[i]['ORIGIN_DATE'], flight_excel.loc[i]['ORIGIN_TIME'], flight_excel.loc[i]['DEST_DATE'], flight_excel.loc[i]['DEST_TIME'],
                                                              flight_excel.loc[i]['FLIGHT_DURATION'], flight_excel.loc[i]['AIRCRAFT_TYPE']))
    for j in range(i, max_row_number) :
        if (visited[j]) :
            continue

        origin_dest_equal = (flight_temp_subset.origin == flight_excel.loc[j]['ORIGIN']) and (flight_temp_subset.dest == flight_excel.loc[j]['DEST'])
        type_equal = (flight_temp_subset.flight_type == flight_excel.loc[j]['AIRCRAFT_TYPE'])

        basic_origin2minute = getMinute(flight_temp_subset.origin_date.year, flight_temp_subset.origin_date.month, flight_temp_subset.origin_date.day, flight_temp_subset.origin_time.hour, flight_temp_subset.origin_time.minute)
        basic_dest2minute = getMinute(flight_temp_subset.dest_date.year, flight_temp_subset.dest_date.month, flight_temp_subset.dest_date.day, flight_temp_subset.dest_time.hour, flight_temp_subset.dest_time.minute) 
        flight_origin2minute = getMinute(flight_excel.loc[j]['ORIGIN_DATE'].year, flight_excel.loc[j]['ORIGIN_DATE'].month, flight_excel.loc[j]['ORIGIN_DATE'].day, flight_excel.loc[j]['ORIGIN_TIME'].hour, flight_temp_subset.dest_time.minute)
        flight_dest2minute = getMinute(flight_excel.loc[j]['DEST_DATE'].year, flight_excel.loc[j]['DEST_DATE'].month, flight_excel.loc[j]['DEST_DATE'].day, flight_excel.loc[j]['DEST_TIME'].hour, flight_excel.loc[j]['DEST_TIME'].minute)
        time_possible = (abs(flight_origin2minute - basic_origin2minute) <= 240) and (abs(basic_dest2minute - flight_dest2minute) <= 240)
        
        if (not origin_dest_equal or not type_equal or not time_possible) :
            continue
        
        visited[j] = True
        flight_temp_subset.append(Flight(flight_excel.loc[j]['INDEX'], flight_excel.loc[j]['ORIGIN'], flight_excel.loc[j]['DEST'],
                                                              flight_excel.loc[j]['ORIGIN_DATE'], flight_excel.loc[j]['ORIGIN_TIME'], flight_excel.loc[j]['DEST_DATE'], flight_excel.loc[j]['DEST_TIME'],
                                                              flight_excel.loc[j]['FLIGHT_DURATION'], flight_excel.loc[j]['AIRCRAFT_TYPE']))
    flight_subset_list.append(flight_temp_subset)

print(len(flight_subset_list))
for temp in flight_subset_list :
    flight_name = list()
    for tflight in temp.subset : 
        flight_name.append(tflight.index)
    print(temp.index, len(temp.subset), flight_name)

671
FS1 1 ['F1']
FS2 1 ['F2']
FS3 1 ['F3']
FS4 1 ['F4']
FS5 1 ['F5']
FS6 1 ['F6']
FS7 1 ['F7']
FS8 1 ['F8']
FS9 1 ['F9']
FS10 1 ['F10']
FS11 1 ['F11']
FS12 1 ['F12']
FS13 1 ['F13']
FS14 1 ['F14']
FS15 1 ['F15']
FS16 1 ['F16']
FS17 1 ['F17']
FS18 1 ['F18']
FS19 1 ['F19']
FS20 1 ['F20']
FS21 1 ['F21']
FS22 1 ['F22']
FS23 1 ['F23']
FS24 1 ['F24']
FS25 1 ['F25']
FS26 1 ['F26']
FS27 1 ['F27']
FS28 1 ['F28']
FS29 1 ['F29']
FS30 1 ['F30']
FS31 1 ['F31']
FS32 1 ['F32']
FS33 1 ['F33']
FS34 1 ['F34']
FS35 1 ['F35']
FS36 1 ['F36']
FS37 1 ['F37']
FS38 1 ['F38']
FS39 1 ['F39']
FS40 1 ['F40']
FS41 1 ['F41']
FS42 1 ['F42']
FS43 1 ['F43']
FS44 1 ['F44']
FS45 1 ['F45']
FS46 1 ['F46']
FS47 1 ['F47']
FS48 1 ['F48']
FS49 1 ['F49']
FS50 1 ['F50']
FS51 1 ['F51']
FS52 1 ['F52']
FS53 1 ['F53']
FS54 1 ['F54']
FS55 1 ['F55']
FS56 1 ['F56']
FS57 1 ['F57']
FS58 1 ['F58']
FS59 1 ['F59']
FS60 1 ['F60']
FS61 1 ['F61']
FS62 1 ['F62']
FS63 1 ['F63']
FS64 1 ['F64']
FS65 1 ['F65']
FS66 1 ['F66']
FS67 1 ['F67']
FS68 1 ['

In [20]:
# Graph insert Node
for i in range(len(flight_subset_list)) :
    airports_set.add(flight_subset_list[i].origin)
    airports_set.add(flight_subset_list[i].dest)
    
for airports in airports_set :
    flight_graph[airports] = list()


# Graph insert Edge
for i in range(len(flight_subset_list)) :
    flight_graph[flight_subset_list[i].origin].append(Flight(flight_subset_list[i].index, flight_subset_list[i].origin, flight_subset_list[i].dest,
                                                              flight_subset_list[i].origin_date, flight_subset_list[i].origin_time, flight_subset_list[i].dest_date, flight_subset_list[i].dest_time,
                                                              flight_subset_list[i].flight_duration, flight_subset_list[i].flight_type))

In [21]:
pairing_set = list()

# clustering을 통한 flights set의 크기를 줄여야지 경우의 수가 줄어듬
# 하지만 그렇다면 시간을 어떻게 배분해야하는가? 기준은? 시간 → 기종? 조금더 생각해봐야 할 거 같다.

# 여기서 Node 시작에 따라 끝나는 경우는
# 만나는 Base가 같은경우가 있고,
# 만약 끝이 Base가 아닌데 갈 곳이 없는 경우가 있다.
# 그렇다면 모든 pairing의 모든 경우를 넣어도 모든 경우가 나온다!
# 

for airport in airports_set :
    # pairing_set.append(["-" * 10, airport, "-" * 10])

    for flight in flight_graph[airport] :
        # 항공편, 항공편의 순서를 넣는다
        stack_temp = [Stackitem(flight)]
        while stack_temp:
            # 현재 진행 중인 항공편의 정보
            flights_information_now = stack_temp.pop()

            # 만약 해당 항공편이 진행됨으로서 base로 간다면 해당 항공편은 더 진행할 필요가 없음
            if (flights_information_now.flight.dest == flight.origin) :
                pairing_set.append(copy.deepcopy(flights_information_now.flights_sequence))
                continue

            # Search Space Extend
            for flight_next in flight_graph[flights_information_now.flight.dest] :
                # 항공기 기종이 같은지? 시간이 가능한지? (날짜가 같다면 시간까지 비교, 다르다면 날짜 비교)
                flight_type_equal = (flights_information_now.flight.flight_type == flight_next.flight_type)
                flight_time_possible = (flights_information_now.flight.dest_date == flight_next.origin_date and flights_information_now.flight.dest_time < flight_next.origin_time) or (flights_information_now.flight.dest_date < flight_next.origin_date)
                flight_day_possible = ((flight_next.dest_date.month - flight.origin_date.month) * 30 + flight_next.dest_date.day  - flight.origin_date.day <= 144) 
                flight_landing_possible = (flights_information_now.landing_number <= 3)

                # 기종이 다르거나 시간이 불가능하면 해당 항공편은 Search에 사용하지 않음
                if (not flight_type_equal or not flight_time_possible or not flight_day_possible or not flight_landing_possible) :
                    continue

                # 기종이 같고, 시간이 가능하다면 Search를 위해 stack에 추가(항공편의 순서까지)
                temp_list = copy.deepcopy(flights_information_now.flights_sequence)
                if (flights_information_now.flight.dest_date < flight_next.origin_date) :
                    for i in range(0, int(flight_next.origin_date.day - flights_information_now.flight.dest_date.day)) :
                        temp_list.append("Layover")
                        
                stack_temp.append(Stackitem(flight_next, temp_list, flights_information_now.landing_number))

for pairing in pairing_set :
    print(pairing)

# 첫번째 방안 : 모든 항공편들의 순서를 저장한다
# 그렇다면 만들 수 있는(DeadHead, Layover, Feasible) Pairing의 모든 경우가 나온다.
# 하지만 이럴때는 시간이 n^n까지 나올 수 있는 것으로 보인다.
# 여기서 경우의 수를 줄이는 조건을 넣을 때
# 첫번째 - 항공기 종류
# 두번째 - 전 항공편의 도착시간보다 현 항공편의 출발시간이 느려야한다
# 세번째 - Crew의 Trun Time이 MMT보다 크거나 같아야 한다.(Slack Time은 어떻게 될려나?)
# feasible pairing은 좀 많다. -> 그럼 DeadHead가 존재하는 pairing은 넣어야하는 것인가?
# 지연전파? 존재한다?

# stack에 넣을 때 모든 정보를 넣어버린다면? 오히려 쉬워진다. 하지만 메모리를 많이 쓰므로 최적화가 필요함(조건을 생각해야 함)

['FS2', 'Layover', 'Layover', 'Layover', 'FS27']
['FS2', 'Layover', 'Layover', 'FS26']
['FS2', 'Layover', 'FS25']
['FS2', 'FS24']
['FS3', 'Layover', 'Layover', 'FS27']
['FS3', 'Layover', 'FS26']
['FS3', 'FS25']
['FS4', 'Layover', 'FS27']
['FS4', 'FS26']
['FS5', 'FS27']
['FS28', 'Layover', 'FS64']
['FS28', 'FS63']
['FS29', 'Layover', 'Layover', 'Layover', 'Layover', 'Layover', 'Layover', 'FS67']
['FS29', 'FS64']
['FS29', 'Layover', 'Layover', 'Layover', 'FS54', 'Layover', 'Layover', 'FS9', 'FS67']
['FS29', 'Layover', 'Layover', 'Layover', 'FS54', 'FS6', 'Layover', 'Layover', 'FS67']
['FS29', 'Layover', 'Layover', 'Layover', 'Layover', 'Layover', 'FS49', 'FS9', 'FS67']
['FS29', 'Layover', 'Layover', 'Layover', 'Layover', 'FS45', 'Layover', 'FS9', 'FS67']
['FS29', 'Layover', 'Layover', 'FS44', 'Layover', 'Layover', 'Layover', 'FS9', 'FS67']
['FS29', 'Layover', 'Layover', 'FS44', 'Layover', 'FS6', 'Layover', 'Layover', 'FS67']
['FS29', 'FS42', 'Layover', 'FS23', 'Layover', 'Layover', 'Layo